In [ ]:
# Initial Imports
import pandas as pd
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np

# Import S&P 500 stock ticker list
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_stocks = table[0]
df_stocks.head()
df_stocks.to_csv('S&P500-Info.csv')
df_stocks.to_csv('S&P500-Symbols.csv', columns=['Symbol'])
df_tickers = df_stocks['Symbol']
df_tickers.head()

In [ ]:
load_dotenv()

In [ ]:
# Import historical price performance for S&P 500 stocks over the last 7 years
timeframe = '1Day'
start_date = pd.Timestamp('2016-06-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2023-04-30', tz='America/New_York').isoformat()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

alpaca = tradeapi.REST(
   alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

tickers = df_stocks['Symbol'][:500].tolist()

# tickers = df_stocks['Symbol'][101:200].tolist()

df_sp500_hist = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

In [ ]:
df_sp500_hist

In [ ]:
# test test tes

In [ ]:
# Date column Normalisation
df_sp500_hist = df_sp500_hist.reset_index()

df_sp500_hist = df_sp500_hist.rename(columns={
    "timestamp": "date"
})

#  index_col="Date", infer_datetime_format=True, parse_dates=True)

In [ ]:
# Delete Unnecessary Columns

df_sp500_hist = df_sp500_hist.drop(columns=["open", "high", "low", "volume", "trade_count", "vwap"])
df_sp500_hist

In [ ]:
df_sp500_hist.set_index('date', inplace=True)


In [ ]:
df_sp500_hist.head()

In [ ]:
df_sp500_hist['Daily Return'] = df_sp500_hist.groupby('symbol')['close'].pct_change()

In [ ]:
display(df_sp500_hist.head())
display(df_sp500_hist.tail())

In [ ]:


# Calculate Daily Returns
# df_sp500_returns = df_sp500.pct_change()
# df_sp500_returns.head(100)


# Drop nulls
df_sp500_hist.isnull()
df_sp500_hist.isnull().sum()

df_sp500_hist = df_sp500_hist.dropna().copy()
df_sp500_hist

# Rename `Close` Column to be specific to this portfolio.
# df_sp500_returns = df_sp500_returns.rename(columns={
#     "close": "daily_returns"
# })

df_sp500_hist.head()



In [ ]:
# Annualized Sharpe Ratios

## 1 sharpe_ratios = ((combined_df.mean()-combined_df['rf_rate'].mean()) * 252) / (combined_df.std() * np.sqrt(252))
##  sharpe_ratios

## 2  sharpe_ratios = (all_returns.mean() * 252) / (all_portfolio_std * np.sqrt(252))
## 2 sharpe_ratios.head()

# joined_std = joined_portfolios.std()

sharpe_ratios = (df_sp500_hist.mean() * 252) / (df_sp500_hist.std * np.sqrt(252))
sharpe_ratios